# Bagging simplificado

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_friedman1
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

In [ ]:
#Creación del ejemplo de regresión
#https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_friedman1.html
n_points = 200 # points
x, y = make_friedman1(n_samples=n_points, n_features=10,noise=1, random_state=1)

In [ ]:
#Separación de la muestra en entrenamiento y prueba
tam_test = 0.2
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=tam_test, random_state=0)

In [ ]:
#Función error cuadrático medio.
def Fun_ECM(v1,v2):
    suma, n = 0, len(v1)
    for i in range(n):
        suma = suma + (v1[1]-v2[1])**2
    return (suma**0.5)/n

# Regresión Árbol Binario

In [ ]:
#Implementación del árbol binario
regTree = DecisionTreeRegressor(random_state=1)
regTree.fit(x_train,y_train)
yhatAB = regTree.predict(x_test)
print("El error cuadrático medio es igual a ",Fun_ECM(yhatAB,y_test))

# Gráfica comparativa

In [ ]:
yhatAB_aux = np.copy(y_train).tolist()
y_aux = np.copy(y_train).tolist()
yhatAB_aux.extend(yhatAB)
y_aux.extend(y_test)
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1)
plt.plot(range(1,len(yhatAB_aux)+1),yhatAB_aux,color="black",linewidth=1.5,label="Árbol binario")
plt.plot(range(1,len(y_aux)+1),y_aux,color="red",ls="--",linewidth=1.0,label="Serie verdadera")
plt.axvline(x=n_points-tam_test*n_points,color="green",linewidth=3)
plt.legend(loc="best")
plt.show()

In [ ]:
#Construcción de bagging
n_estimators=10
bag = np.empty((n_estimators), dtype=object) #Construcción de un array con entradas vacias

In [ ]:
#Bootstraping (notar como el muestreo se realiza en las bases de entrenamiento)
for i in range(n_estimators):
    ids = np.random.choice(range(0,len(x_train)),size=int(len(x_train)),replace=True) #array en donde hay x_test valores escogidos aleatoriamente de range(0,len(x_test))
    np.unique(ids) #del arreglo, unique regresa (de manera ordenada de menor a mayor) los valores de manera única
    x_boot = x_train[np.unique(ids)] #del array x_train se seleecionan los renglones (vectores) indicados por los índice ids
    y_boot = y_train[np.unique(ids)] #del array y_train se seleecionan los renglones (vectores) indicados por los índice ids
    bag[i] = DecisionTreeRegressor(random_state=1)
    bag[i].fit(x_boot,y_boot)
#Predicción por bagging (dado cada ajuste en la muestra se pronóstica en el conjunto de prueba)
yhatbag = np.zeros(len(y_test))
for i in range(n_estimators):
    yhatbag = yhatbag + bag[i].predict(x_test)
yhatbag = yhatbag/n_estimators
print("El error cuadrático medio es igual a ",Fun_ECM(yhatbag,y_test))

In [ ]:
yhatAB_aux = np.copy(y_train).tolist()
y_aux = np.copy(y_train).tolist()
yhatAB_aux.extend(yhatbag)
y_aux.extend(y_test)
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1)
plt.plot(range(1,len(yhatAB_aux)+1),yhatAB_aux,color="black",linewidth=1.5,label="Árbol binario")
plt.plot(range(1,len(y_aux)+1),y_aux,color="red",ls="--",linewidth=1.0,label="Serie verdadera")
plt.axvline(x=n_points-tam_test*n_points,color="green",linewidth=3)
plt.legend(loc="best")
plt.show()

# Bosque aleatorio

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10,max_features=5,random_state=1)
rf.fit(x_train,y_train)
yhatBA = rf.predict(x_test)
print("El error cuadrático medio es igual a ",Fun_ECM(yhatBA,y_test))

In [ ]:
yhatBA_aux = np.copy(y_train).tolist()
y_aux = np.copy(y_train).tolist()
yhatBA_aux.extend(yhatBA)
y_aux.extend(y_test)
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1)
plt.plot(range(1,len(yhatBA_aux)+1),yhatBA_aux,color="black",linewidth=1.5,label="Bosque aleatorio")
plt.plot(range(1,len(y_aux)+1),y_aux,color="red",ls="--",linewidth=1.0,label="Serie verdadera")
plt.axvline(x=n_points-tam_test*n_points,color="green",linewidth=3)
plt.legend(loc="best")
plt.show()

# Ejemplo 1. Ajuste de una curva no lineal

In [ ]:
#Ejemplo parábola
M = pd.read_csv("C:\\Users\\ricardo\\Desktop\\ArbolesBinarios\\EjemploCircunferencia.csv",dtype='str',encoding = "ISO-8859-1")
M[["Y","X"]] = M[["Y","X"]].astype(float)
M.head()

In [ ]:
#Árbol binario
x, y = np.asarray(M["X"]), np.asarray(M["Y"])
x = x.reshape(-1,1)
y = y.reshape(-1,1)
regTree = DecisionTreeRegressor(random_state=1)
regTree.fit(x,y)
yhatAB3 = regTree.predict(x)
print("El error cuadrático medio es igual a ",Fun_ECM(yhatAB3,y))

In [ ]:
#Regresión lineal
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(x,y)
yhatRL2 = lin_reg.predict(x)
print("El error cuadrático medio es igual a ",Fun_ECM(yhatRL2,y))

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1)
plt.plot(x,yhatAB3,color="orange",linewidth=1.5,label="Árbol binario")
plt.plot(x,yhatRL2,color="blue",linewidth=1.5,label="Regresión lineal")
plt.plot(x,y,color="red",ls="--",linewidth=1.0,label="Serie verdadera")
plt.legend(loc="best")
plt.show()

# Ejercicio 1. De manera simimilar al ejercicio anterior, realizar un comparativo entre el ajuste que proporciona un árbol binario respecto a una regresión lineal con los datos del archivo "EjemploParabola.csv".